Setup PyTorch for Apple Silicon GPU

In [1]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

mps


Start of training

In [2]:
from transformers import DebertaV2Tokenizer, DebertaV2Model

model_name = "microsoft/deberta-v3-base"
tokenizer = DebertaV2Tokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=False, use_fast=True)
model = DebertaV2Model.from_pretrained(model_name).to(device)

Dataset load

In [3]:
import pandas as pd
import swifter

In [4]:
ARTIFACTS_BASE = '../../artifacts/step-1/classic-ml-ms-deberta-v3-base/'

In [5]:
# Function to convert string representation of vector to list of floats
import numpy as np

def convert_to_float_list(s):
    s = s.strip('[]')
    return np.array(s.split(), dtype=float)

In [6]:
test_df = pd.read_csv(ARTIFACTS_BASE + 'test_df.csv')
test_df['embedding'] = test_df['embedding'].swifter.apply(convert_to_float_list)
test_df.head()

Pandas Apply:   0%|          | 0/65416 [00:00<?, ?it/s]

,prompt,label,embedding
0,"Task: Given a list of words, identify which wo...",0,"[0.0470528901, 0.128666952, 0.120663926, 0.011..."
1,So I was watching episode 11 of Dark Angel sea...,0,"[-0.077564396, 0.109083176, 0.141840488, 0.140..."
2,"Given the sentence ""Lots of people standing on...",0,"[0.0886795521, 0.168291405, -0.0653214753, 0.0..."
3,"In this task, you are given the name of an Ind...",0,"[-0.00887174066, 0.0103212046, -0.156578302, 0..."
4,"Task: Replace all the sentences that use ""i"" w...",0,"[0.0129042789, 0.0639195219, -0.259266078, 0.2..."


In [7]:
train_df = pd.read_csv(ARTIFACTS_BASE + 'train_df.csv')
train_df['embedding'] = train_df['embedding'].swifter.apply(convert_to_float_list)
train_df.head()

Pandas Apply:   0%|          | 0/261738 [00:00<?, ?it/s]

,prompt,label,embedding
0,My question is: Alani earned $45 for 3 hours o...,0,"[-0.0976323485, 0.15560329, 0.0484190285, -0.0..."
1,"Q: Yes / no, is the following a verifiable fac...",0,"[0.0271683801, 0.0977859125, -0.032584101, -0...."
2,Read a code snippet and detect if there are an...,0,"[-0.00371995103, 0.226242572, 0.0596871153, 0...."
3,"As a highly advanced chatbot, it is your duty ...",1,"[0.276234031, 0.0609874725, -0.398053914, 0.04..."
4,Tell me the main idea of this paragraph.,0,"[-0.0443628579, 0.187087953, -0.0733531415, -0..."


In [8]:
X_test = pd.DataFrame(np.array(test_df['embedding'].to_list()))
y_test = test_df["label"]

In [9]:
X_train = pd.DataFrame(np.array(train_df['embedding'].to_list()))
y_train = train_df["label"]

In [10]:
# Check number of training and testing samples
print(f"#Training Samples: {len(X_train)}")
print(f"#Testing Samples: {len(X_test)}")

#Training Samples: 261738
#Testing Samples: 65416


In [11]:
# Import classification models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

# Initialize estimators using their default parameters
estimators = [
    ("Naive Bayes", GaussianNB()),
    ("Logistic Regression", LogisticRegression()),
    ("Support Vector Machine", svm.SVC()),
    ("Random Forest", RandomForestClassifier())
]

In [ ]:
# Import performance metrics libraries
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Prepare a DataFrame to keep track of the models' performance
results = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1 score"])

# Iterate through each estimator in the list
for est_name, est_obj in estimators:
    
    # Fit the model
    est_obj.fit(X_train, y_train)
    
    # Use the model to predict unseen prompts
    y_predict = est_obj.predict(X_test)
    
    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_predict)
    precision = precision_score(y_test, y_predict)
    recall = recall_score(y_test, y_predict)
    f1 = f1_score(y_test, y_predict)  
    
    # Store performance metrics
    results.loc[est_name] = [accuracy, precision, recall, f1]

/Users/nr/Documents/LPNU/masters/llm-prompt-injection-detector/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
